<a href="https://colab.research.google.com/github/sarbaniAi/Cloud-AI-ML/blob/main/News_Recommendation_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative Filtering: User Based Recommendations

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

### Load plat_cont_en_prsnt Data

In [ ]:
#plat_cont_en_prsnt = pd.read_csv('news_genres_final.csv')
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
consumer_tx = pd.read_csv("/content/drive/MyDrive/consumer_transanctions.csv")

In [ ]:
consumer_tx.head()

In [ ]:
consumer_tx.shape

In [ ]:
consumer_tx.columns

In [ ]:
platform_content = pd.read_csv("/content/drive/MyDrive/platform_content.csv")

In [ ]:
print(platform_content.shape)
print(platform_content.columns)


In [ ]:
pd.set_option('max_colwidth', None)

In [ ]:
platform_content.head()

### Use the ‘platform_content’ data to extract the data set that contains the English articles present on the platform.

In [ ]:
platform_content["language"].value_counts()

In [ ]:
platform_content.info()

In [ ]:
platform_content["interaction_type"].value_counts()

In [ ]:
platform_content["item_type"].value_counts()

 - I have to ensure that the system does not recommend any news article that has been pulled out from the app or has already been seen by the user.

 - In addition, only the articles that are written in the English language must be considered for content-based recommendations.

### Dataset -> plat_cont_en_prsnt


 - The final generated list must contain the names of the recommended articles, along with their IDs.

In [ ]:
plat_cont_en_prsnt = platform_content[(platform_content["language"] == "en") & (platform_content["interaction_type"] == "content_present")]

print(plat_cont_en_prsnt.shape)

### Data pre-processing consumer transactionn data -> user content data 

#### Extract Rating

The data set provided to you does not contain user ratings that could be used to generate recommendations. For that you need to impute the rating values based on the feature ‘interaction type’ with the highest weightage to content_followed, followed by content_commented_on, content saved, content liked and content_watched. The final data set ‘consumer_interactions’ that will be used to build the recommendation system should contain a feature called ‘ratings’ along with other features that are required to build the ALS, user-based and item-based collaborative filtering models.

In [ ]:
print(consumer_tx.info())
print("***************\n")
print(consumer_tx["interaction_type"].value_counts())

In [ ]:
consumer_tx.groupby(["consumer_id", "item_id"]).count()


* content_watched         1
* content_liked          2 
* content_saved         3  
* content_commented_on   4  
* content_followed     5

In [ ]:
cleanup_nums = {"interaction_type": {"content_watched": 1, "content_liked": 2,  "content_saved": 3, "content_commented_on": 4, "content_followed": 5,}}

consumer_tx["interaction_type"] = consumer_tx.replace(cleanup_nums)["interaction_type"];

In [ ]:
consumer_tx.head()

In [ ]:
print(consumer_tx["consumer_id"].value_counts())
print(consumer_tx["item_id"].value_counts())

In [ ]:
consumer_rating = consumer_tx.groupby(['consumer_id', 'item_id' ], as_index = False).agg({'interaction_type':'mean'}).reset_index().round({'interaction_type': 0})

#consumer_rating.to_csv("test.csv")


In [ ]:
consumer_rating[consumer_rating["consumer_id"] == -9223121837663643404]

## Load Ratings Data

In [ ]:
#ratings = pd.read_csv('ratings.csv')

In [ ]:
consumer_rating.shape

### Extracting consumer id in seprate table

In [ ]:
consumer_id_tbl = consumer_rating["consumer_id"]

In [ ]:
consumer_id_tbl_un = consumer_id_tbl.unique()

### create unique of consumer od

In [ ]:
consumer_id_tbl_un

In [ ]:

consumer_rating.consumer_id = pd.Categorical(consumer_rating.consumer_id)


In [ ]:
consumer_rating["consumer_code"] = consumer_rating.consumer_id.cat.codes

In [ ]:
consumer_rating.head()

In [ ]:
consumer_rating.item_id = pd.Categorical(consumer_rating.item_id)
consumer_rating["item_code"] = consumer_rating.item_id.cat.codes


In [ ]:
consumer_rating.head()

In [ ]:
consumer_rating.drop(['index'], axis=1, inplace=True)

In [ ]:
n_users = consumer_rating.consumer_id.nunique()

n_items = consumer_rating.item_id.nunique()

In [ ]:
n_users

In [ ]:
n_items

In [ ]:
n_users * n_items

## Dividing the dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(consumer_rating, test_size=0.30, random_state=31)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
consumer_rating.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()

In [ ]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train['interaction_type'] = dummy_train['interaction_type'].apply(lambda x: 0 if x>=1 else 1)

In [ ]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot(
    index='consumer_code',
    columns='item_code',
    values='interaction_type'
).fillna(1)

## Create empty data matrix: user*news

In [ ]:
data_matrix = np.zeros((n_users, n_items))


In [ ]:
data_matrix

In [ ]:
data_matrix.shape   

## Fill user*news Train matrix with rating values

In [ ]:
for line in train.itertuples():
    #print(line)
    #print(line[3])
    #print(line[4]) 
    #print(data_matrix[line[1]-1, line[2]-1])
    #print(line[3])
    data_matrix[line[4], line[5]] = line[3]


In [ ]:
data_matrix

In [ ]:
data_matrix.shape

## Create Data Matrix with Test Data

In [ ]:
data_matrix_test = np.zeros((n_users, n_items))

In [ ]:
data_matrix_test

In [ ]:
for line in test.itertuples():
    data_matrix_test[line[4], line[5]] = line[3]

In [ ]:
data_matrix_test.shape

In [ ]:
data_matrix_test

### Create Data Matrix with full data

In [ ]:
data_matrix_full = np.zeros((n_users, n_items))


In [ ]:
for line in consumer_rating.itertuples():
    data_matrix_full[line[4], line[5]] = line[3]


In [ ]:
data_matrix_full

In [ ]:
data_matrix_full.shape

# Pairwise Distance

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances 

In [ ]:

user_similarity = 1- pairwise_distances(data_matrix, metric='cosine')


In [ ]:
user_similarity

In [ ]:
user_similarity.shape

### Here, we are not removing the NaN values and calculating the mean
only for the movies rated by the user. Will be used later after user based prediction 

In [ ]:
data_matrix.shape

In [ ]:
data_matrix.T

In [ ]:
data_matrix.T.shape

## plat_cont_en_prsnt pairwise similarity distance

In [ ]:
item_similarity = 1-pairwise_distances(data_matrix.T, metric='cosine')

In [ ]:
item_similarity

In [ ]:
item_similarity.shape

## Dot product of Data Matrix with similarity matrix

In [ ]:
item_prediction = np.dot(user_similarity, data_matrix_test)

In [ ]:
item_prediction

In [ ]:
item_prediction.shape

In [ ]:
prediction_df = pd.DataFrame(item_prediction)

In [ ]:
prediction_df.head(1000)

In [ ]:
prediction_df.shape

## Get recommended news articles for specific user 

In [ ]:
prediction_df.iloc[1313]

### Sample Test with random consumer_id/Consumer_code

In [ ]:
consumer_rating.loc[(consumer_rating["consumer_code"] == 1313) & (consumer_rating["interaction_type"] == 3)]

In [ ]:
platform_content.loc[platform_content["item_id"] == -3662111110237135231	]

## Do not recommend any news which is already rated by the user

### The original list of news alreday rated by the user =1313 -> 
original_recommend_by_usr

In [ ]:
original_recommend_by_usr = consumer_rating[(consumer_rating["consumer_code"] == 1313)]["item_code"].unique()

In [ ]:

original_recommend_by_usr

### Prediction of top 10 News Items for specific consumer id 

In [ ]:
#sort top 10 news items
prediction_df.iloc[1313].sort_values(ascending=False)[:10]

In [ ]:
recommended_news_df = pd.DataFrame(prediction_df.iloc[1313].sort_values(ascending=False))

In [ ]:
recommended_news_df.head()

In [ ]:
recommended_news_df.reset_index(inplace=True)

In [ ]:
recommended_news_df.head()

### The final recommendations after removing the news which are already rated by the user = 1313

In [ ]:
recommended_news_df[~ recommended_news_df.iloc[:,0].isin(original_recommend_by_usr)].head()
#recommended_news_df.iloc[:,0]

In [ ]:
#random test with item_code
consumer_rating.loc[(consumer_rating["item_code"] == 844)]

In [ ]:
 #creating dataset with item_Code & item_id mapping
 consum_item_cd_id = consumer_rating[['item_code', 'item_id']]

In [ ]:
consum_item_cd_id

In [ ]:
consum_item_cd_id.loc[(consum_item_cd_id["item_code"] == 844)]

In [ ]:
consum_item_cd_id.shape

In [ ]:
# sorting by item_code
consum_item_cd_id.sort_values("item_code", inplace = True)
 
# dropping ALL duplicate values
consum_item_cd_id.drop_duplicates(subset ="item_code", keep = 'first', inplace = True)
 
# displaying data
consum_item_cd_id.shape

In [ ]:
#test with sample item_code
consum_item_cd_id.loc[(consum_item_cd_id["item_code"] == 844)]


In [ ]:
consum_item_cd_id.shape

### Prediction of top 10 news articles with title & description from platform_content dataframe

In [ ]:
recommended_news_df.columns = ['item_code', 'score']

In [ ]:
recommended_news_top10 = recommended_news_df.head(10)

In [ ]:
recommended_news_top10


## Merge news id with news title

In [ ]:
merged = pd.merge(recommended_news_top10, consum_item_cd_id, on='item_code', how='inner')

In [ ]:
merged


In [ ]:
merged

### Recommemend the news which are in english and currently present in the platform ->

use the dataframe plat_cont_en_prsnt for merging the item_id

In [ ]:
#merged = pd.merge(merged, platform_content, on='item_id', how='left')

merged = pd.merge(merged, plat_cont_en_prsnt, on='item_id', how='left') 

### Since we are interested only in the news  not rated by the user, we will ignore the news rated by the user by making it zero.

In [ ]:

#user_similarity[np.isnan(user_similarity)] = 0
#print(user_similarity)

In [ ]:
user_similarity.shape

In [ ]:
merged.columns

In [ ]:
merged[['item_id','title', 'score']].head(10)

## Item-based collaborative filtering

#### Find the item-similarity matrix based on a similarity measure.

#### Generate the top 10 similar and relevant items based on the similarity scores.

In [ ]:
n_users = consumer_rating.consumer_code.unique().shape[0]
n_items = consumer_rating.item_code.unique().shape[0]
n_items = consumer_rating['item_code'].max()
A = np.zeros((n_items, n_users))

print(n_items)
print(n_users)

for line in consumer_rating.itertuples():
    A[line[5]-1, line[4]-1] = line[3]

print("Original rating matrix : ",A)

In [ ]:
for i in range(len(A)):
  for j in range(len(A[0])):
    if A[i][j]>=3:
      A[i][j]=1
    else:
      A[i][j]=0

In [ ]:
from scipy.sparse import csr_matrix
csr_sample = csr_matrix(A)
print(csr_sample)

In [ ]:
type(csr_sample)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=3, n_jobs=-1)
knn.fit(csr_sample)

In [ ]:
consum_sort_des = consumer_rating.sort_values(['consumer_code'], ascending=[True])
filter1 = consum_sort_des[consum_sort_des['consumer_code'] == 10].item_code
filter1 = filter1.tolist()
filter1 = filter1[:20]
print("Items liked by user: ",filter1)

In [ ]:
csr_sample.shape

In [ ]:
distances1=[]
indices1=[]
for i in filter1:
  distances , indices = knn.kneighbors(csr_sample[i],n_neighbors=3)
  indices = indices.flatten()
  indices= indices[1:]
  indices1.extend(indices)
print("Items to be recommended: ",indices1)

## Content-based filtering

#### Use text processing to analyse the ‘text_description’ feature in the data set.

#### Recommend similar items based on the TF-IDF scores.

In [ ]:
plat_cont_en_prsnt.columns

In [ ]:
plat_cont_en_prsnt['text_description'].str.len()

In [ ]:
np.mean(plat_cont_en_prsnt['text_description'].str.len())

In [ ]:
number_of_text_description = []
for text_description in plat_cont_en_prsnt['text_description']:
    n_text_description = len(text_description.split(','))
    number_of_text_description.append(n_text_description)


In [ ]:
number_of_text_description

In [ ]:
sum(number_of_text_description)

## EDA & Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:

plt.hist(number_of_text_description)
plt.xlabel('distribution of number of words in plat_cont_en_prsnt text_description column')
plt.ylabel('number of plat_cont_en_prsnt')
plt.show()

Selected DF

In [ ]:
plat_cont_en_prsnt = plat_cont_en_prsnt[['title', 'text_description']]

In [ ]:
plat_cont_en_prsnt.head()

### Create numpy array from text_description column

In [ ]:
text_description_array = plat_cont_en_prsnt['text_description'].to_numpy()

In [ ]:
text_description_array

In [ ]:
text_description_array[0]

In [ ]:
type(text_description_array[0])

### Generate Words from text_description Array by Splitting

In [ ]:
words_list = []
for keyword in text_description_array:
    splitted_words = keyword.lower().split(',')
    words_list.append(splitted_words)
    

In [ ]:
words_list

In [ ]:
words_list[7]

In [ ]:
len(words_list), len(words_list[0]), len(words_list[1])

## Create Dictionary

Install Gensim

In [ ]:
! pip install gensim

In [ ]:
from gensim.corpora.dictionary import Dictionary

create a dictionary of words from words list

In [ ]:
dictionary = Dictionary(words_list) 

In [ ]:
dictionary

In [ ]:
len(dictionary)

#### Total number of words in the words_list

In [ ]:
number_words = 0
for word in words_list:
    number_words = number_words + len(word)

In [ ]:
number_words

In [ ]:
dictionary.get(0), dictionary.get(1), dictionary.get(100)

In [ ]:
dictionary.get(1000)

## Generate Bag of Words

In [ ]:
words_list[0]

doc2bow = Document to BOW[Bag of word]

In [ ]:
bow = dictionary.doc2bow(words_list[0])

In [ ]:
bow

In [ ]:
len(words_list[0]), len(bow)

### Generate Corpus by creating BOW of each document

In [ ]:

corpus = [dictionary.doc2bow(doc) for doc in words_list] 


In [ ]:
print(corpus)

In [ ]:
len(corpus), len(corpus[0]), len(corpus[1])

In [ ]:
len(words_list), len(words_list[0])

### TF-IDF Model

TF = Term frequency -> count/no.

IDF = Inverse doc freq. -> log(N/n)

In [ ]:

from gensim.models.tfidfmodel import TfidfModel


create tfidf model of the corpus

In [ ]:

tfidf = TfidfModel(corpus)

In [ ]:
tfidf

In [ ]:
tfidf[corpus[0]]

## Generate Similarity Matrix

In [ ]:
from gensim.similarities import MatrixSimilarity


In [ ]:
sims = MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))


In [ ]:
print(sims)

In [ ]:
sims[corpus[0]]

In [ ]:
len(sims[corpus[0]])

## Generate Recommendation for a news

In [ ]:
news_title = "IEEE to Talk Blockchain at Cloud Computing Oxford-Con - CoinDesk"

#### n = no of recommendations

In [ ]:
n = 5

#### Get the news row

In [ ]:
news = plat_cont_en_prsnt.loc[plat_cont_en_prsnt.title==news_title] 

In [ ]:
news

#### Generate words_list by splitting the text_description column

In [ ]:
words_list = news['text_description'].iloc[0].split(',')

In [ ]:
words_list

#### set the query_doc to the words_list

In [ ]:
query_doc = words_list 

In [ ]:
query_doc

#### get bag of words from the query_doc

In [ ]:
query_doc_bow = dictionary.doc2bow(query_doc) 

In [ ]:
query_doc_bow

In [ ]:
len(query_doc_bow), len(query_doc)

### Generate TF-IDF values for the query_doc_bow

In [ ]:
query_doc_tfidf = tfidf[query_doc_bow] 

In [ ]:
query_doc_tfidf

### Get Similarity Score using Similarity Matrix

In [ ]:
similarity_array = sims[query_doc_tfidf] 

In [ ]:
similarity_array

In [ ]:
len(similarity_array)

### Create a Series to Visualize Similarity Score along with news Names

In [ ]:
similarity_series = pd.Series(similarity_array.tolist(), index=plat_cont_en_prsnt.title.values)

In [ ]:
similarity_series

### Sort the series to get Top Recommended plat_cont_en_prsnt

In [ ]:

recommended_plat_cont_en_prsnt = similarity_series.sort_values(ascending=False)[1:n+1] 



In [ ]:
recommended_plat_cont_en_prsnt

## Get highest TF-IDF values words for this news which influenced the Recommendations

In [ ]:
news

### Corpus for 16th index

In [ ]:
corpus[16]

In [ ]:
corpus[news.index.values.tolist()[0]]

### TF-IDF Values

In [ ]:
tfidf[corpus[news.index.values.tolist()[0]]]

### Sorted TF-IDF values

In [ ]:

sorted_tfidf_weights = sorted(tfidf[corpus[news.index.values.tolist()[0]]], key=lambda w: w[1], reverse=True)


In [ ]:
sorted_tfidf_weights

In [ ]:
dictionary.get(3515)

In [ ]:
dictionary.get(3534)

In [ ]:
print('Top words associated with this news by tf-idf are: ')
for term_id, weight in sorted_tfidf_weights[:5]:
    print(" '%s' %.5f" %(dictionary.get(term_id), weight))



# Creating a Modular code by creating a function for Query news Input

In [ ]:
def news_recommendation(news_title, number_of_hits=5):
    news = plat_cont_en_prsnt.loc[plat_cont_en_prsnt.title==news_title] # get the news row
    text_description = news['text_description'].iloc[0].split(',') #get the text_description as a Series (news['text_description']),
    # get just the text_description string ([0]), and then convert to a list of text_description (.split(',') )
    query_doc = text_description #set the query_doc to the list of text_description
    
    query_doc_bow = dictionary.doc2bow(query_doc) # get a bag of words from the query_doc
    query_doc_tfidf = tfidf[query_doc_bow] #convert the regular bag of words model to a tf-idf model where we have tuples
    # of the news ID and it's tf-idf value for the news

    similarity_array = sims[query_doc_tfidf] # get the array of similarity values between our news and every other news. 
    #So the length is the number of plat_cont_en_prsnt we have. To do this, we pass our list of tf-idf tuples to sims.

    similarity_series = pd.Series(similarity_array.tolist(), index=plat_cont_en_prsnt.title.values) #Convert to a Series
    recommended_plat_cont_en_prsnt = similarity_series.sort_values(ascending=False)[1:number_of_hits+1] 
    #get the top matching results, i.e. most similar plat_cont_en_prsnt; start from index 1 because every news is most similar to itself
    print("Top Recommended plat_cont_en_prsnt along with Similarity Score: ")
    print(recommended_plat_cont_en_prsnt)

In [ ]:
news_recommendation('Google Data Center 360° Tour', 5)

In [ ]:
news_recommendation('Google Data Center 360° Tour', 5)

In [ ]:
news_recommendation('Google Data Center 360° Tour', 5)

## ALS 

### Create Sparse User-Item Matrix

In [ ]:
consumer_rating_cpy = consumer_rating.copy()

In [ ]:
consumer_rating_cpy.shape

In [ ]:
consumer_rating_cpy.head()
print(consumer_rating_cpy)

In [ ]:
consumer_rating_cpy.drop(["consumer_id", "item_id"], axis =1)

In [ ]:
consumer_rating_cpy.describe()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
alpha = 40

In [ ]:
consumer_rating_cpy.shape[0]

In [ ]:
#sparse_user_item = csr_matrix( ([alpha]*consumer_rating_cpy.shape[0], (consumer_rating_cpy['consumer_code'], consumer_rating_cpy['item_code']) ))
sparse_user_item = csr_matrix( (consumer_rating_cpy['interaction_type'].astype(float), (consumer_rating_cpy['consumer_code'], consumer_rating_cpy['item_code']) ))

In [ ]:
print(sparse_user_item)

### shape : 1895x2987.  since Total No of consumers  = 1895, No of News = 2987

### Convert to Array

In [ ]:
csr_user_array = sparse_user_item.toarray()

In [ ]:
csr_user_array

In [ ]:
len(csr_user_array), len(csr_user_array[0]), csr_user_array[1][1]

In [ ]:
max(csr_user_array[1])

### csr matrix only stores where value is 40 [non-zero]. (Compressed Sparse Row)

In [ ]:
print(sparse_user_item)

### Create item-user sparse matrix

In [ ]:
sparse_item_user = sparse_user_item.T.tocsr()

In [ ]:
print(sparse_item_user)



```
# This is formatted as code
```

### shape : 2987x1895.  since Total No of News  = 2987 & No of users = 1895

In [ ]:
csr_item_array = sparse_item_user.toarray()

In [ ]:
csr_item_array

In [ ]:
len(csr_item_array), len(csr_item_array[0]), csr_item_array[1][2]

In [ ]:
print(sparse_item_user)

## Create train, test data

In [ ]:

!pip install implicit

In [ ]:
from implicit.evaluation import train_test_split

In [ ]:
train, test = train_test_split(sparse_item_user, train_percentage=0.8)

In [ ]:
print(train)

In [ ]:
print(test)

## Building ALS Model

In [ ]:
import implicit

In [ ]:
##! pip install implicit

In [ ]:
#model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, iterations=20, calculate_training_loss=False)
model = implicit.als.AlternatingLeastSquares(factors=50)

In [ ]:
print(model)

### Train the Model

In [ ]:
data_conf = (sparse_item_user * alpha).astype('double')

In [ ]:
print(data_conf)

In [ ]:
#model.fit(train)
model.fit(data_conf)

In [ ]:
item_factors, user_factors = model.item_factors, model.user_factors

In [ ]:
item_factors.shape

In [ ]:
item_factors[0]

In [ ]:
user_factors.shape

In [ ]:
user_factors[0]

## Generating recommendations for a user_id

In [ ]:
consumer_code = 1
print(sparse_user_item)

In [ ]:
model.recommend(consumer_code, sparse_user_item)

In [ ]:
model.recommend(consumer_code, sparse_user_item, N=20)

In [ ]:
output = model.recommend(consumer_code, sparse_user_item)

In [ ]:
output

In [ ]:
output_df = pd.DataFrame(output, columns=['item_code', 'als_score'])

In [ ]:
output_df

# Load News Data

In [ ]:
news = platform_content[['item_id', 'title']]

In [ ]:
news.head()

# Merge recommendation output with news Data

In [ ]:
merged = pd.merge(output_df, consum_item_cd_id, how='left', on='item_code')

In [ ]:
merged

In [ ]:
consum_item_cd_id

## Generating recommendations for news


In [ ]:

item_Code = 11
n_similar = 10

In [ ]:

similar = model.similar_items(item_Code, n_similar)


In [ ]:
similar

In [ ]:
type(similar)

In [ ]:
similar[0]

In [ ]:
similar_df = pd.DataFrame(similar, columns=['item_code', 'score'])

In [ ]:
similar_df

In [ ]:
news

# Merge recommendation output with news Data

In [ ]:
merged_similar = pd.merge(similar_df, consum_item_cd_id, how='left', on='item_code')

In [ ]:
merged_similar

In [ ]:
merged_similar = pd.merge(merged_similar, news, how='left', on='item_id')

In [ ]:
#plat_cont_en_prsnt

## Hybrid recommendation system

#### Normalise the scores for content and collaborative filtering and combine them with an appropriate weightage to build a hybrid model.

#### Try out hybrids of different types of models that can help recommend items similar to a particular item. 

#### For example, Content+Item-based collaborative model, ALS+Item-based collaborative model, ALS+Content-based model, etc.

### Hybrid Recommendation System: Combine Collaborative & Content based

In [ ]:
data_matrix = consumer_rating.pivot(
    index='consumer_code',
    columns='item_code',
    values='interaction_type'
).fillna(0)

In [ ]:
data_matrix.head()

In [ ]:
data_matrix.shape

# Pairwise Distance

## news pairwise similarity distance

In [ ]:
data_matrix.T

In [ ]:
data_matrix.T.shape

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances 

In [ ]:

item_similarity = 1- pairwise_distances(data_matrix.T, metric='cosine')


In [ ]:
item_similarity

In [ ]:
item_similarity.shape

## Get similar recommended news for a item_id

In [ ]:
news_index = 5

In [ ]:
news_prediction = pd.DataFrame(item_similarity)

In [ ]:
news_prediction.head()

In [ ]:
news_prediction.shape

In [ ]:
news_prediction.iloc[news_index]

In [ ]:
item_recommendation = pd.DataFrame(news_prediction.iloc[news_index].sort_values(ascending=False))

In [ ]:
item_recommendation.head()

In [ ]:
item_recommendation.reset_index(inplace=True)

In [ ]:
item_recommendation.head()

In [ ]:
item_recommendation.columns = ['item_code', 'score']

In [ ]:
item_recommendation.head()

In [ ]:
item_recommendation.dtypes

In [ ]:
item_recommendation.describe()

In [ ]:
merged = pd.merge(item_recommendation, consum_item_cd_id, on='item_code', how='inner')

In [ ]:
merged


In [ ]:
merged

In [ ]:
merged = pd.merge(merged, platform_content, on='item_id', how='left')

In [ ]:
merged

In [ ]:
merged.shape

In [ ]:
collab_output = merged[merged['title'].notna()]

In [ ]:
collab_output.shape

In [ ]:
collab_output.head(10)

In [ ]:
collab_output.describe()

### Min Max Normalization

In [ ]:
collab_output['collaborative_score_normalized'] = (collab_output['score']-min(collab_output['score']))/(max(collab_output['score'])-min(collab_output['score']))

In [ ]:
collab_output.head()

In [ ]:
collab_output.describe()

In [ ]:
collab_output.shape

In [ ]:
collab_output['title'][:10]

# Content Based Recommendations

In [ ]:
platform_content.columns

In [ ]:
news_keywords = platform_content[['title', 'text_description']]

In [ ]:
news_keywords.columns

## convert to numpy arrays

In [ ]:
keywords_array = news_keywords['text_description'].to_numpy()

In [ ]:
keywords_array

In [ ]:
len(keywords_array)

In [ ]:
news_keywords.shape

In [ ]:
keywords_array[0]

## Split keywords & create word list

In [ ]:
words_list = []

for keyword in keywords_array:
    
    splitted_words = str(keyword).lower().split()
    
    words_list.append(splitted_words)

In [ ]:
words_list

In [ ]:
len(words_list), len(words_list[0]), len(words_list[1])

## Create Dictionary, BOW, tfidf model & Similarity Matrix

In [ ]:
from gensim.corpora.dictionary import Dictionary

# create a dictionary from words list
dictionary = Dictionary(words_list) 

#create corpus where the corpus is a bag of words for each document
corpus = [dictionary.doc2bow(doc) for doc in words_list] 

from gensim.models.tfidfmodel import TfidfModel
#create tfidf model of the corpus
tfidf = TfidfModel(corpus) 

from gensim.similarities import MatrixSimilarity
# Create the similarity matrix. This is the most important part where we get the similarities between the news.
sims = MatrixSimilarity(tfidf[corpus], num_features=len(dictionary))


In [ ]:
print(sims)



```
# This is formatted as code
```

## For an input news, Run the Model & return Recommended news

In [ ]:
def news_recommendation(news_title):
    # get the news row
    news = news_keywords.loc[news_keywords.title==news_title] 
    
    #split the keywords
    keywords = news['text_description'].iloc[0].split() 

    #set the list of words to query_doc
    query_doc = keywords 
    
    # get a bag of words from the query_doc
    query_doc_bow = dictionary.doc2bow(query_doc) 

    #convert the regular bag of words model to a tf-idf model
    query_doc_tfidf = tfidf[query_doc_bow] 

    # get similarity values between input news and all other news
    similarity_array = sims[query_doc_tfidf] 

    #Convert to a Series
    similarity_series = pd.Series(similarity_array.tolist(), index=news_keywords.title.values) 

    #get the most similar news 
    similarity_output = similarity_series.sort_values(ascending=False)
    return similarity_output

In [ ]:
news_keywords["title"]

In [ ]:
content_output = news_recommendation('JPMorgan Software Does in Seconds What Took Lawyers 360,000 Hours')

In [ ]:
content_output

In [ ]:
content_output[:10]

In [ ]:
type(content_output)

In [ ]:
content_df = pd.DataFrame(content_output)

In [ ]:
content_df.head()

In [ ]:
content_df.reset_index(inplace=True)

In [ ]:
content_df.columns = ['title', 'score']

In [ ]:
content_df.head()

In [ ]:
content_df['content_score_normalized'] = (content_df['score']-min(content_df['score'])) / (max(content_df['score']) - min(content_df['score']))

In [ ]:
content_df.head()

In [ ]:
content_df.describe()

In [ ]:
content_df.shape

# Hybrid: Combine two Models

In [ ]:
hybrid_output = pd.merge(collab_output,content_df, how='left', on='title')

In [ ]:
hybrid_output.head()

In [ ]:
hybrid_output.shape

In [ ]:
hybrid_output['final_score'] = (hybrid_output['collaborative_score_normalized'] + hybrid_output['content_score_normalized'])/2

In [ ]:
hybrid_output.head()

In [ ]:
hybrid_output = hybrid_output[['title','final_score']]

In [ ]:
hybrid_output.head()

In [ ]:
hybrid_output.sort_values(by='final_score', ascending=False)[:10]

In [ ]:
hybrid_output.shape

## Model evaluation

Use appropriate evaluation metrics, such as RMSE, MAE and precision@k, to evaluate the recommendations generated for a user as mentioned in the first part of the problem statement and use global precision@k to assess the overall performance of the recommendation system.

 

Also, recommend appropriate evaluation techniques for the second problem statement. This is an open-ended question and you can explore the different techniques that are available for online evaluation and give your thoughts on it. You need not come up with the ones which are already covered in the module.

### USER Based Method - Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(consumer_rating, test_size=0.30, random_state=31)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
consumer_rating.shape

In [ ]:
train.head()

In [ ]:
test.head()

## Create empty data matrix: user*news

In [ ]:
data_matrix = np.zeros((n_users, n_items))


In [ ]:
data_matrix

In [ ]:
data_matrix.shape   

In [ ]:
df_pivot = train.pivot(
    index='consumer_code',
    columns='item_code',
    values='interaction_type'
).fillna(0)

df_pivot.head(3)

In [ ]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()

In [ ]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train['interaction_type'] = dummy_train['interaction_type'].apply(lambda x: 0 if x>=1 else 1)

In [ ]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot(
    index='consumer_code',
    columns='item_code',
    values='interaction_type'
).fillna(1)

In [ ]:
dummy_train.head()

## Using Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_pivot, metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

In [ ]:
user_correlation.shape

## Using adjusted Cosine 

### Here, we are not removing the NaN values and calculating the mean 
only for the movies rated by the user

In [ ]:
# Create a user-movie matrix.
df_pivot = train.pivot(
    index='consumer_code',
    columns='item_code',
    values='interaction_type'
)

In [ ]:
df_pivot.head()

### Normalising the rating of the movie for each user around 0 mean

In [ ]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [ ]:
df_subtracted.head()

### Finding cosine similarity

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the user insead of predicting it for the movie not rated by the user. 

In [ ]:
# Find out the common users of test and train dataset.
common = test[test.consumer_code.isin(train.consumer_code)]
common.shape

In [ ]:
common.head()

In [ ]:
# convert into the user-movie matrix.
common_user_based_matrix = common.pivot_table(index='consumer_code', columns='item_code', values='interaction_type')

In [ ]:
# Convert the user_correlation matrix into dataframe.
user_correlation_df = pd.DataFrame(user_correlation)

In [ ]:
df_subtracted.head(1)

In [ ]:
user_correlation_df['consumer_code'] = df_subtracted.index
user_correlation_df.set_index('consumer_code',inplace=True)
user_correlation_df.head()

In [ ]:
common.head(1)

In [ ]:
list_name = common.consumer_code.tolist()

user_correlation_df.columns = df_subtracted.index.tolist()


user_correlation_df_1 =  user_correlation_df[user_correlation_df.index.isin(list_name)]

In [ ]:
user_correlation_df_1.shape

In [ ]:
user_correlation_df_2 = user_correlation_df_1.T[user_correlation_df_1.T.index.isin(list_name)]

In [ ]:
user_correlation_df_3 = user_correlation_df_2.T

In [ ]:
user_correlation_df_3.head()

In [ ]:
user_correlation_df_3.shape

In [ ]:
user_correlation_df_3[user_correlation_df_3<0]=0

common_user_predicted_ratings = np.dot(user_correlation_df_3, common_user_based_matrix.fillna(0))
common_user_predicted_ratings

In [ ]:
dummy_test = common.copy()

dummy_test['interaction_type'] = dummy_test['interaction_type'].apply(lambda x: 1 if x>=1 else 0)

dummy_test = dummy_test.pivot_table(index='consumer_code', columns='item_code', values='interaction_type').fillna(0)

In [ ]:
dummy_test.shape

In [ ]:
common_user_predicted_ratings = np.multiply(common_user_predicted_ratings,dummy_test)

In [ ]:
common_user_predicted_ratings.head(2)

Calculating the RMSE for only the movies rated by user. For RMSE, normalising the rating to (1,5) range.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_user_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

In [ ]:
common_ = common.pivot_table(index='consumer_code', columns='item_code', values='interaction_type')

In [ ]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [ ]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

# Evaluation - Item Item

Taking the transpose of the rating matrix to normalize the rating around the mean for different movie ID. In the user based similarity, we had taken mean for each user instead of each movie.

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the user insead of predicting it for the movie not rated by the user. 

In [ ]:
df_pivot = train.pivot(
    index='consumer_code',
    columns='item_code',
    values='interaction_type'
).T

df_pivot.head()

In [ ]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [ ]:
df_subtracted.head()

Finding the cosine similarity using pairwise distances approach

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

Filtering the correlation only for which the value is greater than 0. (Positively correlated)

In [ ]:
item_correlation[item_correlation<0]=0
item_correlation

In [ ]:
test.columns

In [ ]:
common =  test[test.item_code.isin(train.item_code)]
common.shape

In [ ]:
common.head(4)

In [ ]:
common_item_based_matrix = common.pivot_table(index='consumer_code', columns='item_code', values='interaction_type').T

In [ ]:
common_item_based_matrix.shape

In [ ]:
item_correlation_df = pd.DataFrame(item_correlation)

In [ ]:
item_correlation_df.head(1)

In [ ]:
item_correlation_df['item_code'] = df_subtracted.index
item_correlation_df.set_index('item_code',inplace=True)
item_correlation_df.head()

In [ ]:
list_name = common.item_code.tolist()

In [ ]:
item_correlation_df.columns = df_subtracted.index.tolist()

item_correlation_df_1 =  item_correlation_df[item_correlation_df.index.isin(list_name)]

In [ ]:
item_correlation_df_2 = item_correlation_df_1.T[item_correlation_df_1.T.index.isin(list_name)]

item_correlation_df_3 = item_correlation_df_2.T

In [ ]:
item_correlation_df_3.head()

In [ ]:
item_correlation_df_3[item_correlation_df_3<0]=0

common_item_predicted_ratings = np.dot(item_correlation_df_3, common_item_based_matrix.fillna(0))
common_item_predicted_ratings


In [ ]:
common_item_predicted_ratings.shape

Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train



In [ ]:
dummy_test = common.copy()

dummy_test['interaction_type'] = dummy_test['interaction_type'].apply(lambda x: 1 if x>=1 else 0)

dummy_test = dummy_test.pivot_table(index='consumer_code', columns='item_code', values='interaction_type').T.fillna(0)

common_item_predicted_ratings = np.multiply(common_item_predicted_ratings,dummy_test)

The products not rated is marked as 0 for evaluation. And make the item- item matrix representaion.


In [ ]:
common_ = common.pivot_table(index='consumer_code', columns='item_code', values='interaction_type').T

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_item_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

In [ ]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [ ]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

### The RMSE value is not showing much difference in User based & Item Based recommendation system. 
However by checking the user based recommendations result I am finalizing the hybrid Model - user based & content based recommendation system.